# Solving a Complex Capacity Expansion Model with Benders decomposition

_**[Power Systems Optimization](https://github.com/east-winds/power-systems-optimization)**_

We demonstrate the application of Benders Decomposition to solve the capacity expansion model illustrated in [Complex Capacity Expanion](07-Complex-Capacity-Expansion.ipynb).

In the `complex_expansion_data/` path, we provide several different sets of inputs using different sample time periods, for your use and experimentation, including 10 days (used as default below), 4 weeks, 8 weeks, 16 weeks, and 52 weeks (full 8760 hours). Alter the `inputs_path` parameter below to select a different time series if desired.

## LOAD PACKAGES

In [ ]:
# Ensure these packages are installed; if not, use the Pkg package and Pkg.add() function to install
using Pkg; Pkg.activate(dirname(@__DIR__))

using JuMP
using HiGHS
using Plots
using DataFrames, CSV
import Printf

include("benders_cems.jl")

  Activating project at `~/Code/power-systems-optimization`


print_iteration (generic function with 1 method)

## SELECT INPUTS PATH

In [2]:
# Read input data for a case with 10 sample days of data
inputs_path = "complex_expansion_data/10_days/"


"complex_expansion_data/10_days/"

## LOAD INPUTS

In [3]:
inputs = load_inputs(inputs_path);

Scale inputs to improve numerics:

In [4]:

    scaling!(inputs)

## GENERATE MONOLITHIC MODEL

In [5]:
Expansion_Model = generate_monolithic_model(inputs)

A JuMP Model
├ solver: HiGHS
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 15956
├ num_constraints: 58279
│ ├ AffExpr in MOI.EqualTo{Float64}: 1497
│ ├ AffExpr in MOI.GreaterThan{Float64}: 480
│ ├ AffExpr in MOI.LessThan{Float64}: 40800
│ ├ VariableRef in MOI.GreaterThan{Float64}: 15476
│ └ VariableRef in MOI.LessThan{Float64}: 26
└ Names registered in the model
  └ :cCapEnergyNew, :cCapEnergyOld, :cCapNew, :cCapOld, :cDemandBalance, :cMaxCharge, :cMaxFlow, :cMaxNSE, :cMaxPower, :cMaxSOC, :cMinFlow, :cRampDown, :cRampDownWrap, :cRampUp, :cRampUpWrap, :cSOC, :cSOCWrap, :cTransCap, :eFixedCostsGeneration, :eFixedCostsStorage, :eFixedCostsTransmission, :eNSECosts, :eVariableCosts, :vCAP, :vCHARGE, :vE_CAP, :vFLOW, :vGEN, :vNEW_CAP, :vNEW_E_CAP, :vNEW_T_CAP, :vNSE, :vRET_CAP, :vRET_E_CAP, :vRET_T_CAP, :vSOC, :vT_CAP

### RUN MODEL

In [6]:
@time optimize!(Expansion_Model)

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e-04, 4e+00]
  Cost   [1e-02, 6e+02]
  Bound  [8e-01, 9e+01]
  RHS    [4e-03, 4e+04]
Presolving model
37442 rows, 14141 cols, 110872 nonzeros  0s
37438 rows, 14141 cols, 110864 nonzeros  0s
Presolve : Reductions: rows 37438(-5339); columns 14141(-1815); elements 110864(-12532)
Solving the presolved LP
IPX model has 37438 rows, 14141 columns and 110864 nonzeros
Input
    Number of variables:                                14141
    Number of free variables:                           480
    Number of constraints:                              37438
    Number of equality constraints:                     1442
    Number of matrix entries:                           110864
    Matrix range:                                       [1e-04, 4e+00]
    RHS range:                                          [4e-03, 9e+01]
    Objective range:                                    [1

## GENERATE PLANNING MODEL

In [7]:
planning_model = generate_planning_model(inputs);

## GENERATE OPERATION MODELS

In [8]:
operation_models = generate_decomposed_operation_models(inputs);

In [9]:
@time benders_solution = benders_iterations(100,planning_model,operation_models);

Iteration  Lower Bound  Upper Bound          Gap
        1   0.0000e+00   3.8163e+06          Inf
        2   4.9144e+02   3.5552e+06   7.2333e+03
        3   5.8966e+02   4.4835e+04   7.5036e+01
        4   8.3687e+02   4.2195e+04   4.9420e+01
        5   1.4867e+03   1.7511e+04   1.0779e+01
        6   4.8120e+03   1.7511e+04   2.6390e+00
        7   4.8819e+03   1.7511e+04   2.5869e+00
        8   4.9457e+03   1.7511e+04   2.5407e+00
        9   5.6965e+03   1.3259e+04   1.3276e+00
       10   5.9492e+03   1.3259e+04   1.2288e+00
       11   7.0844e+03   1.0938e+04   5.4397e-01
       12   8.2119e+03   1.0938e+04   3.3198e-01
       13   8.9282e+03   1.0938e+04   2.2512e-01
       14   9.0843e+03   1.0768e+04   1.8530e-01
       15   9.2946e+03   1.0151e+04   9.2093e-02
       16   9.6568e+03   1.0043e+04   3.9958e-02
       17   9.7195e+03   1.0043e+04   3.3257e-02
       18   9.7400e+03   1.0043e+04   3.1079e-02
       19   9.7711e+03   1.0034e+04   2.6866e-02
       20   9.7993e+

┌ Info: Convergence tolerance satisfied!
└ @ Main /Users/fpecci/Code/power-systems-optimization/Notebooks/benders_cems.jl:530
